In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import csv
import time
from sklearn.cluster import KMeans
from scipy.spatial import distance
import numpy as np
from sklearn.neighbors import KNeighborsClassifier


In [0]:
!cd "/content/drive/My Drive/Colab Notebooks/"

In [9]:
!ls

drive  sample_data


In [13]:
csvfile = open('/content/drive/My Drive/Colab Notebooks/train_labels.csv','rU')
csvfile2 = csv.reader(csvfile, delimiter=',') 
train_labels=[]
for num1 in list(csvfile2)[0]:
    k=int(num1)
    train_labels.append(k)
csvfile.close()

csvfile1 = open('/content/drive/My Drive/Colab Notebooks/test_labels.csv','rU')
csvfile3 = csv.reader(csvfile1, delimiter=',') 
test_labels=[]
for num2 in list(csvfile3)[0]:
    k=int(num2)
    test_labels.append(k)
csvfile.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 'U' mode is deprecated
  if __name__ == '__main__':


In [0]:
def getFeaturesAndRegions(name,n):
    train_features = []
    detectedRegions = []
    for i in range(n):
        file = '/content/drive/My Drive/Colab Notebooks/' + name +'_sift_features/' + str(i+1) + '_' + name + '_sift.csv'
        csvFile = csv.reader(open(file))
        regions = 0
        for total_line in csvFile:
            train_features.append(total_line[4:])
            regions += 1
        detectedRegions.append(regions)
    return detectedRegions , train_features

In [0]:
train_size = 1888
test_size = 800
train_regions,train_features = getFeaturesAndRegions('train' , train_size)
test_regions,test_features = getFeaturesAndRegions('test' , test_size)

In [0]:
def getDistance(first,clusterCenters):
    f1=[]
    for x in first:
        k=float(x)
        f1.append(k)

    dist = []
    for i in clusterCenters:
        distance = np.linalg.norm(i-f1)
        dist.append(distance)
    return dist
def BagofVisualWords(siftFeatures,train_regions,clusterCenters,clusters,bool_):
    if(bool_ == 1):
        name = "train"
    if(bool_ == 0):
        name = "test"
    vocab = [] 
    features = [0] * clusters
    Sum = 0
    temp = 0
    for feature in siftFeatures:
        dist = getDistance(feature,clusterCenters)
        i = dist.index(min(dist))
        features[i] += 1
        Sum += 1
        if Sum == train_regions[temp]:
            vocab.append(features)
            features = [0] * clusters
            Sum = 0
            temp += 1
    return vocab

In [17]:
clusters = 16
kmeans = KMeans(n_clusters = clusters)
kmeans.fit(train_features)
cluster_centers = kmeans.cluster_centers_
print(cluster_centers)

[[35.62751495 16.15945121  9.63621533 ... 16.42832643 10.29137909
  17.85803321]
 [17.59868918  7.09183714  6.20591857 ... 13.46192651  9.423714
  13.16174777]
 [17.00438926 23.46379849 25.41978726 ... 33.19953339 59.0011072
  48.0194551 ]
 ...
 [28.39389844 10.22311257  5.99618652 ... 17.9560726  28.50690288
  37.54537556]
 [22.07004919 14.10728951 12.88303452 ... 17.55986502 15.45420173
  12.93031671]
 [27.51824793 10.81535228  5.66649987 ... 12.07752869 10.43171204
  17.0040032 ]]


In [0]:
vocab_Train = BagofVisualWords(train_features,train_regions,cluster_centers,clusters,1)
vocab_Test = BagofVisualWords(test_features,test_regions,cluster_centers,clusters,0)

In [20]:
k_nn = 5
knn_model = KNeighborsClassifier(n_neighbors = k_nn)
knn_model.fit(vocab_Train, train_labels)
test_prediction = knn_model.predict(vocab_Test)

from sklearn.metrics import classification_report
print (classification_report(test_labels, test_prediction, target_names=[ str(i) for i in list(range(1,9))]))


              precision    recall  f1-score   support

           1       0.36      0.48      0.41       100
           2       0.72      0.87      0.79       100
           3       0.45      0.38      0.41       100
           4       0.60      0.40      0.48       100
           5       0.42      0.44      0.43       100
           6       0.30      0.35      0.33       100
           7       0.45      0.45      0.45       100
           8       0.49      0.39      0.44       100

   micro avg       0.47      0.47      0.47       800
   macro avg       0.48      0.47      0.47       800
weighted avg       0.48      0.47      0.47       800



In [21]:
from sklearn.metrics import accuracy_score
categorization_accuracy = accuracy_score(test_labels, test_prediction)
print (categorization_accuracy)

0.47
